# Class 1: Solving the Poisson Problem
**Author**: ZongYi Ju  
**Date**: 2024/11/13

---

## Problem Statement

To solve the Poisson problem of finding a function \( u \) that satisfies:


\begin{aligned}
    - \Delta u &= f && \text{in the unit square } \Omega = [0,1] \times [0,1], \\
    u &= 0 && \text{on the bottom and right parts of the boundary}, \\
    \frac{\partial u}{\partial n} &= 0 && \text{on the remaining boundary parts}.
\end{aligned}


## Solution Approach in NGSolve

In NGSolve, we would solve this problem by following these steps:

1. Define the domain $\Omega = [0,1] \times [0,1]$
2. Specify the function space, typically an \( H^1 \)-conforming space, suitable for Poisson problems.
3. Apply the boundary conditions: Dirichlet condition on the bottom and right parts, and Neumann condition on the remaining parts of the boundary.
4. Set up the weak form of the Poisson equation (integration by parts to convert the strong form into a weak form suitable for finite element methods).
5. Solve the resulting linear system to find the approximate solution \( u \) over the unit square.


## 1. import necessray package

In [3]:
from ngsolve import *
from ngsolve.webgui import Draw

## 2. define the domain $\Omega = [0,1] \times [0,1]$ (generate unstructured mesh)

In [7]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2)) # generate mesh h <= 0.2
# unit_square:generate square mesh
print(mesh.nv, mesh.ne)# number of vertices,number of elements
Draw(mesh);# webgui draw mesh

39 56


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

## 3. Specify the function space

In [9]:
fes = H1(mesh, order = 2, dirichlet = "bottom|right")
# H1:generate one-order sobolev space H1
# use two-order ploynomial as basic
# set dirichlet conditional on bottom and right
print(fes.ndof) # number of degree of freedom

133


## 4. Set up the weak form of the Poisson equation(test function)

In [13]:
# define trial and test in respective
u = fes.TrialFunction() # trial function
v = fes.TestFunction() # test function
## define trial and test together
# u,v = fes.TnT()
gfu = GridFunction(fes) # Grid Solution

In [16]:
# define respectively
a = BilinearForm(fes) # define binary linear form operater usually differencial operater
a += grad(u) * grad(v) * dx # du*dv*dx i.e. Laplace Operater
a.Assemble() # assemble

f = LinearForm(fes) # define linear Form
f += x * v * dx # x*v*dx i.e. f(non chi-square item/source item)
f.Assemble();

## define together
# a = BilinearForm(grad(u) * grad(v) * dx).Assemble()
# f = LinearForm(x * v * dx).Assemble()

In [17]:
print(a.mat) # Laplace Operater coefficient matrix
print(f.vec) # source item vector

Row 0:   0: 1   4: -0.5   19: -0.5   39: -0.0833333   40: -0.0833333   50: 0.166667
Row 1:   1: 0.828732   7: -0.195774   8: -0.197948   23: -0.43501   41: -0.0360819   42: -0.0364197   43: -0.0656203   59: 0.0687109   61: 0.069411
Row 2:   2: 1   11: -0.5   12: -0.5   44: -0.0833333   45: -0.0833333   69: 0.166667
Row 3:   3: 0.834885   15: -0.164012   16: -0.157951   30: -0.512921   46: -0.0431063   47: -0.0423806   48: -0.0536605   81: 0.0704417   83: 0.0687058
Row 4:   0: -0.5   4: 1.89671   5: -0.546735   19: -0.180532   20: -0.66944   39: 5.55112e-17   40: 0.0833333   49: -0.0544706   50: -0.140436   51: -0.121211   53: 0.145593   91: 0.0871914
Row 5:   4: -0.546735   5: 1.88547   6: -0.352441   20: -0.196284   21: -0.790007   49: -0.0136061   51: 0.104729   52: -0.0481159   53: -0.174674   54: -0.0778481   56: 0.106856   92: 0.10266
Row 6:   5: -0.352441   6: 1.75375   7: -0.265716   21: -0.432048   22: -0.703548   52: -0.0385388   54: 0.097279   55: -0.0537377   56: -0.12226   

## 5. Solve the resulting linear system

In [19]:
gfu.vec.data = a.mat.Inverse(freedofs = fes.FreeDofs()) * f.vec # solve (delta-1 * f)
Draw(gfu); # draw solve
print(gfu.vec)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

       0
       0
       0
 0.0923179
       0
       0
       0
       0
       0
       0
       0
       0
 0.0578988
 0.0863391
 0.0954151
 0.0945036
 0.0888226
 0.0780489
 0.0595559
 0.0331334
 0.043177
 0.0384731
 0.0312766
 0.0180508
 0.0379191
 0.0426386
 0.0472136
 0.0758445
 0.0892927
 0.0932644
 0.0919339
  0.0863
 0.0722526
 0.0576401
 0.0666805
 0.0710587
 0.0846294
 0.0618883
 0.0766818
       0
 -0.00563337
       0
       0
 0.0203536
       0
 -0.0350728
 0.00280938
 -0.00728756
 -0.00137235
       0
 -0.00966674
 -0.0121644
       0
 -0.0238901
 -0.0131488
       0
 -0.0161892
 -0.0200118
 -0.00657505
 -0.0212905
       0
 -0.0255408
 -0.00925383
       0
 -0.0375247
 -0.0131487
       0
 -0.0261275
 -0.0183435
 -0.0333457
 -0.0247246
 -0.0242762
 -0.00473494
 -0.0120835
 -0.0145349
 -0.00640155
 -0.0108854
 -0.00549118
 -0.00937973
 -0.00705122
 -0.0052206
 -0.00304656
 -0.00758425
 0.00107111
 0.00110145
 -0.00824163
 0.000388213
 0.00186718
 -0.0082305
 -0.00056295